In [ ]:
# 4개 열 병합 -> 2만개 까지 cosine_similarity 가능, 전체 병합 -> 만개까지 가능
# 중복된 값 너무 많이나와서 제거함 -> 데이터가 50만개 에서 59000개 됨

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
data = pd.read_excel("./datasets/복합기 크롤링 중복제거.xlsx")

In [14]:
data.head()

,상품명,가격,스펙 목록,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,ApeosPort 2560,2125000,흑백 디지털 복합기,잉크젯,포토프린터,컬러 출력,복사,스캔,팩스,"인쇄 해상도: 4,800dpi",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ApeosPort 3060,2200000,흑백 디지털 복합기,잉크젯,포토프린터,컬러 출력,복사,스캔,팩스,"인쇄 해상도: 4,800dpi",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ApeosPort 3560,2528000,흑백 디지털 복합기,잉크젯,컬러 출력,복사,스캔,"인쇄 해상도: 4,800dpi",무한 패키지,연결방식: USB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ApeosPort C2060,2600000,컬러 디지털 복합기,인쇄,복사,스캔,컬러 속도: 23ppm,흑백 속도: 28ppm,인쇄 해상도: 600 x 600dpi,자동양면인쇄,...,가로: 699mm,세로: 742mm,높이: 759mm,무게: 121kg,NaN,NaN,NaN,NaN,NaN,NaN
4,ApeosPort C2560,2778170,컬러 디지털 복합기,인쇄,복사,스캔,팩스,컬러 속도: 23ppm,흑백 속도: 25ppm,인쇄 해상도: 1200 x 1200dpi,...,무게: 125kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
data.shape

(5901, 30)

In [16]:
data = data.fillna(0)

In [17]:
data.head(2)

,상품명,가격,스펙 목록,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,ApeosPort 2560,2125000,흑백 디지털 복합기,잉크젯,포토프린터,컬러 출력,복사,스캔,팩스,"인쇄 해상도: 4,800dpi",...,0,0,0,0,0,0,0,0,0,0
1,ApeosPort 3060,2200000,흑백 디지털 복합기,잉크젯,포토프린터,컬러 출력,복사,스캔,팩스,"인쇄 해상도: 4,800dpi",...,0,0,0,0,0,0,0,0,0,0


In [18]:
#나누어져 있는 스펙 목록 컬럼을 keyword로 합침
cols = ['Unnamed: ' + str(i) for i in range(3,30)]

In [19]:
data['keyword'] = data[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

In [20]:
colss=['스펙 목록', 'keyword']
data['keyword'] = data[colss].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

In [21]:
data['keyword'].head()

0    흑백 디지털 복합기  잉크젯  포토프린터  컬러 출력  복사  스캔  팩스  인쇄 ...
1    흑백 디지털 복합기  잉크젯  포토프린터  컬러 출력  복사  스캔  팩스  인쇄 ...
2    흑백 디지털 복합기  잉크젯  컬러 출력  복사  스캔  인쇄 해상도: 4,800d...
3    컬러 디지털 복합기  인쇄  복사  스캔  컬러 속도: 23ppm  흑백 속도: 2...
4    컬러 디지털 복합기  인쇄  복사  스캔  팩스  컬러 속도: 23ppm  흑백 속...
Name: keyword, dtype: object

In [22]:
# 상품명, 가격, 스펙목록, keyword 남기고 컬럼 제거
data = data.drop(cols, axis=1)

In [23]:
data.head(3)

,상품명,가격,스펙 목록,keyword
0,ApeosPort 2560,2125000,흑백 디지털 복합기,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
1,ApeosPort 3060,2200000,흑백 디지털 복합기,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
2,ApeosPort 3560,2528000,흑백 디지털 복합기,"흑백 디지털 복합기 잉크젯 컬러 출력 복사 스캔 인쇄 해상도: 4,800d..."


In [25]:
data.shape

(5901, 4)

In [26]:
# 1~3단어로 객체를 만들어주고 문자열을 숫자로 바꾸어 벡터화 시킴.
vector = CountVectorizer(ngram_range=(1,3))

In [27]:
vector_keyword = vector.fit_transform(data['keyword'])

In [28]:
# keyword를 코사인 유사도로 유사도 측정
keyword_c_sim = cosine_similarity(vector_keyword, vector_keyword).argsort()[:, ::-1]

In [29]:
keyword_c_sim.shape

(5901, 5901)

In [30]:
#상품 추천
def get_recommend_list(df, title, top=30):
    #특정 상품의 정보를가져온다.
    target_index = df[df['상품명'] == title].index.values
    
    #비슷한 코사인 유사도를 가진 정보를 뽑아 1차원배열로 저장한다.
    sim_index = keyword_c_sim[target_index, :top].reshape(-1)
    #자기 자신은 제외한다.
    sim_index = sim_index[sim_index != target_index]
    
    #dataframe으로 만들고 가장 비슷한 10개의 항목을 return 한다.
    result = df.iloc[sim_index][:10]
    return result

In [31]:
get_recommend_list(data, title='ApeosPort 2560')

,상품명,가격,스펙 목록,keyword
585,Canon iR2206F,786000,흑백 디지털 복합기,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
3614,Kyocera FS-1035MFP/DP,0,흑백 디지털 복합기,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
3655,Kyocera TASKalfa 4012i,2564000,흑백 디지털 복합기,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
618,Canon IR2425,1096380,흑백 디지털 복합기,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
660,Canon iR2525,1649990,흑백 디지털 복합기,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
130,Brother MFC-L6900DW,657720,흑백 디지털 복합기,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
372,Canon IR ADV DX C3725,2593940,컬러 디지털 복합기,컬러 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
41,ApeosPort-V C2275,0,컬러 디지털 복합기,컬러 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
427,Canon IR C3020,1624000,컬러 디지털 복합기,컬러 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
279,Canon IR ADV C3525(III),2806000,컬러 디지털 복합기,컬러 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
